In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np
import re

In [175]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [176]:
# select Noyelle store manually
store_url = 'https://www.auchandrive.fr/nos-rayons'
driver.get(store_url)

In [13]:

main_categories = []

main_category_list = driver.find_element_by_class_name('nav-list').find_elements_by_class_name('nav-list__item')

for main_category in main_category_list:
    name = main_category.find_element_by_tag_name('a').text
    url = main_category.find_element_by_tag_name('a').get_attribute("href")
    main_categories.append({"name":name, 
                          "url": url
                          })
print(len(main_categories))
selected_categories = main_categories[2:-5]
selected_categories



14


[{'name': 'Produits Frais',
  'url': 'https://www.auchandrive.fr/nos-rayons/produits-frais-R3686962'},
 {'name': 'Fruits & légumes',
  'url': 'https://www.auchandrive.fr/nos-rayons/fruits-legumes-R3686964'},
 {'name': 'Le marché',
  'url': 'https://www.auchandrive.fr/nos-rayons/le-marche-R3686965'},
 {'name': 'Surgelés',
  'url': 'https://www.auchandrive.fr/nos-rayons/surgeles-R3686970'},
 {'name': 'Epicerie sucrée',
  'url': 'https://www.auchandrive.fr/nos-rayons/epicerie-sucree-R3686967'},
 {'name': 'Epicerie salée',
  'url': 'https://www.auchandrive.fr/nos-rayons/epicerie-salee-R3686968'},
 {'name': 'Boissons',
  'url': 'https://www.auchandrive.fr/nos-rayons/boissons-R3686969'}]

In [28]:
categories = []


for main_category in selected_categories:
    main_shelter = main_category['name']
    driver.get(main_category['url'])
    
    parent_category_list = driver.find_elements_by_class_name('nav-n2__item')
    
    for parent_category in parent_category_list:
        parent_shelter = parent_category.find_element_by_class_name('nav-n2__link').text
        category_list = parent_category.find_element_by_class_name('nav-n2__list').find_elements_by_tag_name('li')
        for category in category_list:
            child_shelter = category.find_element_by_tag_name('a').text
            url = category.find_element_by_tag_name('a').get_attribute("href")
            
            categories.append({"main_shelter": main_shelter,
                               "parent_shelter": parent_shelter, 
                               "child_shelter": child_shelter,
                               "url": url
                              })     
    
                
len(categories)



286

In [177]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

product_pages = []

def scrape_shelters(categories):
    for category in categories:
        print(category['url'])
        link = category['url']
        driver.get(link)
        sleep(1)
        shelter_main = category['main_shelter']
        shelter_parent = category['parent_shelter']
        shelter_child = category['child_shelter']
        
        try:
            for i in range(5):
                actions = ActionChains(driver)
                actions.move_to_element(driver.find_elements_by_css_selector("article")[-1]).perform() 
        except:
            pass
                
        soup = BeautifulSoup(driver.page_source, 'lxml')  
        
        brands = []
        try:
            for items in soup.find_all('div', class_="facet facet-multi"):
                if items.find('button', class_="facet-brand"):
                    for item in items.find_all('li', class_="facet-multi__list-item"):
                        brands.append(item.find('label').text)
        except:
            pass
        product_pages.append({"url":link, 
                             "shelter_main": shelter_main,
                             "shelter_parent": shelter_parent,
                             "shelter_child": shelter_child,
                             "brands":brands,
                             "soup": soup.select('article')
                            })

scrape_shelters(categories)


https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/laits-et-boissons-lactees-R3690757
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/oeufs-R3690758
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/beurres-R3690759
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/margarines-R3690760
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/cremes-R3690761
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/sauces-et-aides-culinaires-R3690762
https://www.auchandrive.fr/catalog/produits-frais-3686962/cremerie-3686963/boissons-vegetales-R10258421
https://www.auchandrive.fr/catalog/produits-frais-3686962/yaourts-et-desserts-3686313/yaourts-enfants-R3696423
https://www.auchandrive.fr/catalog/produits-frais-3686962/yaourts-et-desserts-3686313/aux-fruits-et-aromatises-R3696421
https://www.auchandrive.fr/catalog/produits-frais-3686962/yaourts-et-dessert

https://www.auchandrive.fr/catalog/fruits-legumes-3686964/fruits-et-legumes-pret-a-lemploi-12466512/legumes-prets-a-consommer-et-purees-R12467675
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/fruits-et-legumes-pret-a-lemploi-12466512/olives-tapenades-et-legumes-grilles-R5968263
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/fruits-et-legumes-pret-a-lemploi-12466512/salades-en-sachet-R3692290
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/fruits-et-legumes-pret-a-lemploi-12466512/sauces-et-epices-R12467678
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/fruits-et-legumes-pret-a-lemploi-12466512/soupes-fraiches-et-gazpacho-R3692291
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/jus-de-fruits-frais-3686317/jus-dorange-R3827524
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/jus-de-fruits-frais-3686317/jus-dagrumes-R3827518
https://www.auchandrive.fr/catalog/fruits-legumes-3686964/jus-de-fruits-frais-3686317/jus-multifruits-e

https://www.auchandrive.fr/catalog/surgeles-3686970/pommes-de-terre-et-legumes-3686342/melanges-de-legumes-nature-R3699127
https://www.auchandrive.fr/catalog/surgeles-3686970/pommes-de-terre-et-legumes-3686342/legumes-cuisines-et-poeles-R3699126
https://www.auchandrive.fr/catalog/surgeles-3686970/pommes-de-terre-et-legumes-3686342/soupes-R12648465
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/steaks-haches-et-boulettes-R3699132
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/nuggets-et-panes-R3699133
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/volailles-et-autres-viandes-R3699131
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/filets-et-paves-natures-R3699134
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/poissons-panes-R3699136
https://www.auchandrive.fr/catalog/surgeles-3686970/viandes-et-poissons-3686343/poissons-cuisi

https://www.auchandrive.fr/catalog/epicerie-salee-3686968/conserves-plats-cuisines-soupes-3686332/foies-gras-R3723264
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/conserves-plats-cuisines-soupes-3686332/croutons-et-accompagnements-R5084662
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/conserves-plats-cuisines-soupes-3686332/soupes-liquides-R10216262
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/conserves-plats-cuisines-soupes-3686332/soupes-deshydratees-R10216263
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/pates-riz-feculents-3686333/pates-R3702489
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/pates-riz-feculents-3686333/riz-R3702490
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/pates-riz-feculents-3686333/purees-R3702492
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/pates-riz-feculents-3686333/couscous-semoules-R3702493
https://www.auchandrive.fr/catalog/epicerie-salee-3686968/pates-riz-feculents-36

In [248]:
products = []
bad_products = []

for page in product_pages:
    for product in page['soup']:
        try:
            name = product['data-name']
        except:
            bad_products.append(product)
            continue
        try:
            price = product['data-price']
        except:
            bad_products.append(product)
            pass
        try:
            ean = product['data-product-id']
        except:
            bad_products.append(product)
            pass
            
        url = 'https://www.auchandrive.fr' + product.find('a', class_='product-item__main')['href']
        image_url = product.find('picture', class_='product-item__image').find('img')['src']
        price_per_unit = product.find('span', class_='product-item__price-per').text        
        store_name = "Auchan"
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand.lower() in name.lower()])
        else:
            brand = ""
        clean_description = name.lower().strip().replace(brand.lower().strip(), '').strip()

        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "description1": clean_description, 
                         "description": name,                        
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                        # "promo_price": promo_price,
                        # "origin": origin,
                         "image_url": image_url,                    
                         "url": url,
                         "ean": ean,
                         "store": store_name
                        # "offer": offer,
                        # "offer_description": offer_description
                        })
 

len(products)
len(bad_products)


280

In [251]:
product_data = pd.DataFrame(products)
product_data = product_data.drop_duplicates()

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_auchan.csv', index = None, header=True)


In [252]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_main, shelter_parent, shelter_child, description1, description, brand, price, price_per_unit, image_url, url, ean, store]
Index: []
